In [11]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

class TargetDistribution(object):
    def __init__(self, m, var):
        #num_chains, N
        self.m = m
        self.var = var
        self.step_size = tf.get_variable(
                name='step_size',
                initializer=lambda: tf.constant(1.,dtype=tf.float64),
                use_resource=True,
                dtype=tf.float64,
                trainable=False)

        
    def logp(self, z_t):
        #N, N
        L = self.var*tf.ones((10,10),dtype=tf.float64)
        P = tfp.distributions.MultivariateNormalTriL(loc=self.m, scale_tril=L[None,None,:,:])
        #s, num_chains
        log_prob = P.log_prob(z_t)
        #num_chains
        return tf.reduce_sum(log_prob,axis=0)
    
    def chain_logp(self):
        
        
        
        hmc = tfp.mcmc.HamiltonianMonteCarlo(
                        target_log_prob_fn=self.logp,
                        num_leapfrog_steps=2,
                        step_size=self.step_size,
                        step_size_update_fn=tfp.mcmc.make_simple_step_size_update_policy(target_rate=0.75))

        #num_chains, N
        q0 = tf.reduce_mean(self.m,axis=0)
        q0 = tf.reshape(q0, (1, -1))

        _, kernel_results = tfp.mcmc.sample_chain(
            num_results=1,
            num_burnin_steps=0,
            current_state=q0,
            kernel=hmc)
        
        posterior_log_prob = tf.reduce_sum(kernel_results.accepted_results.target_log_prob,name='marginal_log_likelihood')

        return posterior_log_prob

with tf.Session(graph=tf.Graph()) as sess:   
    
    var = tf.get_variable('A',shape=(),trainable=True,dtype=tf.float64)
    # works with:
    # m = tf.placeholder(tf.float64,shape=(1, 10))
    # m = tf.placeholder(tf.float64,shape=(1, None))
    # doesn't work with
    m = tf.placeholder(tf.float64,shape=(1, 1, 10))
    
    P = TargetDistribution(m, var)
    logp = P.chain_logp()
    
    opt = tf.train.AdamOptimizer(1e-1).minimize(-logp,var_list=[var])

    init_op = tf.variables_initializer(tf.global_variables())    
    sess.run(init_op)

/home/albert/miniconda3/envs/tectf/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
